In [1]:
%load_ext autoreload
%load_ext dotenv
%autoreload 2
%dotenv ./env_params 

In [2]:
import torch
from pathlib import Path
from tqdm.auto import tqdm
from torch.nn import Module
from torch.optim import Optimizer
from torch.utils.data import DataLoader
from torchvision.transforms import v2 as t
from typing import Callable, Optional, Literal, Any

import sys; sys.path.append("../") if "../" not in sys.path else None
from viz.dataset_plots import plot_segmentation_samples

import logging
from lightning.pytorch.utilities import disable_possible_user_warnings # type: ignore
logging.getLogger("lightning.pytorch").setLevel(logging.ERROR)
disable_possible_user_warnings()

In [13]:
#from datasets.inria_speedtest import InriaImageFolder, InriaHDF5
from datasets.inria import InriaLitData, InriaSegmentation, InriaHDF5
inria_kwargs = {
    "root" : Path.home() / "datasets" / "urban_footprint",
    #"root" : Path.home() / "datasets" / "urban_footprint",
    #"shards": Path.home() / "shards" / "urban_footprint",
    #"root": Path.home() / "shards" / "urban_footprint",
    "test_split": 0.2, "val_split": 0.2, "random_seed": 69,
    "tile_size": (512, 512), "tile_stride": (512, 512),
    "split": "train",
    "shard_size_in_mb": 256 
}
#InriaSegmentation.write_to_litdata(**inria_kwargs)
#ds = InriaHDF5(**inria_kwargs)
InriaSegmentation.write_to_hdf(**inria_kwargs)

train tiled dataset @ [/home/sambhav/datasets/urban_footprint/inria.h5]


In [16]:
#plot_segmentation_samples(ds, 8)

In [ ]:
# TODO: Use Pytorch Profiler Here

from segmentation_models_pytorch import Unet
from geovision.datasets.inria import InriaImageFolder, InriaLitData, InriaHDF5

inria_kwargs = {
    "root" : Path.home() / "datasets" / "urban_footprint", 
    "split": "train",
    "shuffle": True,
    "test_split": 0.2, "val_split": 0.2, "random_seed": 69,
    "tile_size": (512, 512), "tile_stride": (512, 512)
}

dl = DataLoader(
    dataset = InriaHDF5(**inria_kwargs), 
    batch_size = 2, num_workers = 4, 
    pin_memory = True, prefetch_factor = 10 
)
unet = Unet("resnet18", classes=2, encoder_weights="imagenet") 
loss_fn = torch.nn.BCEWithLogitsLoss()
adam = torch.optim.Adam(unet.parameters(), lr = 1e-5)

def train_one_epoch(dataloader: DataLoader, model: Module, criterion: Module, optimizer: Optimizer, limit_train_batches: Optional[int] = None):
    if limit_train_batches is None:
        limit_train_batches = len(dataloader)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.train()
    for idx, batch in tqdm(enumerate(dataloader), total = limit_train_batches, unit = "steps"):
        if idx >= limit_train_batches:
            break
            
        images, masks = batch[0].to(device), batch[1].to(device)
        preds = model(images) 
        loss = criterion(preds.argmax(1).to(torch.float32), masks.argmax(1).to(torch.float32)).mean()
        loss.requires_grad_()
        #print(f"Step: {idx}, Loss: {loss}")
        #print(images.shape, images.dtype, images.min().item(), images.max().item())
        #print(masks.shape, masks.dtype, masks.min().item(), masks.max().item())
        #print(preds.shape, preds.dtype, preds.min().item(), preds.max().item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

train_one_epoch(dl, unet, loss_fn, adam, 500)
torch.cuda.empty_cache()